- 문자, 코드 포인트, 바이트 표현
- 이진 시퀀스의 고유한 특징 : bytes, bytearray, memoryview
- 전체 유니코드 및 레거시 문자셋에 대한 코덱
- 인코딩 에러를 피하고 다루기
- 텍스트 파일을 다룰 때의 모범 사례
- 기본 인코딩 및 표준 입출력 문제
- 정규화를 이용한 안전한 유니코드 텍스트 비교
- 정규화, 케이스 폴딩, 발음 구별 기호 강제 제거를 위한 유틸리티 함수
- locale과 PyUCA 라이브러리를 이용한 유니코드 텍스트의 적절한 정렬
- 유니코드 데이터베이스 안의 문자 메타데이터
- str과 bytes를 다루는 이중모드 API

### 4.1 문자 문제
- 코드 포인트를 바이트로 변환하는 것을 __인코딩__, 바이트를 코드 포인트로 변환하는 것을 __디코딩__ 이라고 한다.

In [9]:
s = 'café'
print (len(s))
b= s.encode('utf8')
print (b)
len(b)
print (b.decode('utf8'))

4
b'caf\xc3\xa9'
café


### 4.2 바이트에 대한 기본 지식

In [21]:
cafe = bytes ('café', encoding='utf8')
print (cafe)
print (cafe[0])
print (cafe[:1])
cafe_arr =  bytearray(cafe)
print(cafe_arr)
print (cafe_arr[-1:])

b'caf\xc3\xa9'
99
b'c'
bytearray(b'caf\xc3\xa9')
bytearray(b'\xa9')


- 화면에 출력 가능한 아스키 문자(공백에서 물결표(~)까지)는 아스키 문자 그대로 출력한다.
- 탭, 개행 문자, 캐리지 리턴, 백슬래시(\)는 이스케이프 시퀀스(\t, \n, \r, \\\\)로 출력한다.
- 그 외의 값은 널 바이트를 나타내는 \x00처럼 16진수 이스케이프 시퀀스로 출력한다.

In [22]:
bytes.fromhex('31 4B CE A9')

b'1K\xce\xa9'

In [23]:
import array
numbers = array.array('h', [-2, -1, 0, 1, 2])
octets = bytes(numbers)
print (octets)

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'


#### 4.2.1 구조체와 메모리 뷰

In [26]:
import struct
fmt = "<3s3sHH"
with open('filter.gif', 'rb') as fp:
    img = memoryview(fp.read())
header = img[:10]
print (bytes(header))
struct.unpack(fmt, header)
del header
del img

FileNotFoundError: [Errno 2] No such file or directory: 'filter.gif'

### 4.3 기본 인코더/디코더

In [45]:
for codec in ['latin_1', 'utf_8', 'utf_16']:
    print(codec, 'El Niño'.encode(codec), sep="\t")
    

latin_1	b'El Ni\xf1o'
utf_8	b'El Ni\xc3\xb1o'
utf_16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


### 4.4 인코딩/디코딩 문제 이해하기
#### 4.4.1 UnicodeEoncdeError 처리하기

In [46]:
city = 'São Paulo'
print (city.encode('utf-8'))
print (city.encode('utf-16'))
print (city.encode('iso8859_1'))
#print (city.encode('cp437'))
print (city.encode('cp437', errors = 'ignore'))
print (city.encode('cp437', errors = 'replace'))
print (city.encode('cp437', errors = 'xmlcharrefreplace'))

b'S\xc3\xa3o Paulo'
b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'
b'S\xe3o Paulo'
b'So Paulo'
b'S?o Paulo'
b'S&#227;o Paulo'


#### 4.4.2 UnicodeDecodeError 처리하기

In [47]:
octets = b'Montr\xe9al'
print (octets.decode("cp1252"))
print (octets.decode("iso8859_7"))
print (octets.decode("koi8_r"))
#print (octets.decode("utf8")) 
print (octets.decode("utf8", errors='replace')) 

Montréal
Montrιal
MontrИal
Montr�al


#### 4.4.3 예상과 달리 인코딩된 모듈을 로딩할 때 발생하는 SyntaxError
#### 4.4.4 바이트 시퀀스의 인코딩 방식을 알아내는 방법
#### 4.4.5 BOM : 유용한 깨진 문자

In [1]:
u16 = 'El Niño'.encode('utf_16')
print (u16)
print (list(u16))
u16le = 'El Niño'.encode('utf_16le')
print (list(u16le))

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'
[255, 254, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]
[69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]


### 4.5  텍스트 파일 다루기
- 텍스트 파일을 읽을 때는 언제나 encoding 인수를 명시적으로 지정해야 한다.

In [13]:
with open ('cafe.txt', 'w', encoding ="utf-8") as wf:
    wf.write('café')

open('cafe.txt', 'r').read()

'café'

#### 4.5.1 기본 인코딩 설정 : 정신 나간거 아냐?

In [14]:
import sys, locale

expressions = """
    locale.getpreferredencoding()
    type(my_file)
    my_file.encoding
    sys.stdout.isatty()
    sys.stdout.encoding
    sys.stdin.isatty()
    sys.stdin.encoding
    sys.stderr.isatty()
    sys.stderr.encoding
    sys.getdefaultencoding()
    sys.getfilesystemencoding()
"""

my_file = open('dummy', 'w')

for expression in expressions.split():
    value = eval(expression)
    print (expression.rjust(30), "->" , repr(value))

 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'UTF-8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


### 4.6 제대로 비교하기 위해 유니코드 정규화하기

In [20]:
s1 = 'café'
s2 = 'cafe\u0301'
print (s1, s2)
print (len(s1), len(s2))
print (s1 ==s2)


café café
4 5
False


In [25]:
from unicodedata import normalize
s1 = 'café'
s2 = 'cafe\u0301'
print (len(s1), len(s2))
print (len(normalize('NFC', s1)), len(normalize('NFC', s2)))
print (len(normalize('NFD', s1)), len(normalize('NFD', s2)))
print (normalize('NFC', s1) == normalize('NFC', s2))
print (normalize('NFD', s1) == normalize('NFD', s2))

4 5
4 4
5 5
True
True


In [28]:
from unicodedata import normalize, name
ohm = '\u2126'
print (name(ohm))
ohm_c = normalize('NFC', ohm)
print (name(ohm_c))
print (ohm == ohm_c)
print (ohm, ohm_c)
print (normalize('NFC', ohm) == normalize('NFC', ohm_c))

OHM SIGN
GREEK CAPITAL LETTER OMEGA
False
Ω Ω
True


In [35]:
from unicodedata import normalize, name
half = '½'
print (normalize('NFKC', half))

four_squared = '42'
print (normalize ('NFKC', four_squared))

micro = 'μ'
micro_kc = normalize('NFKC', micro)
print (micro, micro_kc)
print (ord(micro), ord(micro_kc))
print (name(micro), name(micro_kc))
#μ

1⁄2
42
μ μ
956 956
GREEK SMALL LETTER MU GREEK SMALL LETTER MU


#### 4.6.1 케이스 폴딩
- 모든 텍스트를 소문자로 변환하는 연산, 약간의 변환을 동반한다.

In [40]:
micro = 'μ'
print (name(micro))
micro_cf = micro.casefold()
print (name(micro_cf))
print (micro, micro_cf)
eszett = 'β'
print(name(eszett))
eszett_cf = eszett.casefold()
print (eszett, eszett_cf)

GREEK SMALL LETTER MU
GREEK SMALL LETTER MU
μ μ
GREEK SMALL LETTER BETA
β β


#### 4.6.2 정규화된 텍스트 매칭을 위한 유틸리티 함수

In [1]:
s1 = 'café'
s2 = 'cafe\u0301'
print (s1 == s2)

False


#### 4.6.3 극단적인 '정규화':발음 구별 기호 제거하기

In [2]:
import unicodedata
import string


def shave_marks(txt):
    """Remove all diacritic marks"""
    norm_txt = unicodedata.normalize('NFD', txt)  # <1>
    shaved = ''.join(c for c in norm_txt
                     if not unicodedata.combining(c))  # <2>
    return unicodedata.normalize('NFC', shaved)  # <3>
# END SHAVE_MARKS

# BEGIN SHAVE_MARKS_LATIN
def shave_marks_latin(txt):
    """Remove all diacritic marks from Latin base characters"""
    norm_txt = unicodedata.normalize('NFD', txt)  # <1>
    latin_base = False
    keepers = []
    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:   # <2>
            continue  # ignore diacritic on Latin base char
        keepers.append(c)                             # <3>
        # if it isn't combining char, it's a new base char
        if not unicodedata.combining(c):              # <4>
            latin_base = c in string.ascii_letters
    shaved = ''.join(keepers)
    return unicodedata.normalize('NFC', shaved)   # <5>
# END SHAVE_MARKS_LATIN

# BEGIN ASCIIZE
single_map = str.maketrans("""‚ƒ„†ˆ‹‘’“”•–—˜›""",  # <1>
                           """'f"*^<''""---~>""")

multi_map = str.maketrans({  # <2>
    '€': '<euro>',
    '…': '...',
    'Œ': 'OE',
    '™': '(TM)',
    'œ': 'oe',
    '‰': '<per mille>',
    '‡': '**',
})

multi_map.update(single_map)  # <3>


def dewinize(txt):
    """Replace Win1252 symbols with ASCII chars or sequences"""
    return txt.translate(multi_map)  # <4>


def asciize(txt):
    no_marks = shave_marks_latin(dewinize(txt))     # <5>
    no_marks = no_marks.replace('ß', 'ss')          # <6>
    return unicodedata.normalize('NFKC', no_marks)  # <7>
# END ASCIIZE


### 4.7 유니코드 텍스트 정렬하기

In [5]:
import locale
locale.setlocale(locale.LC_COLLATE, 'pt_BR.UTF-8')
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted_fruits = sorted(fruits, key=locale.strxfrm)
print (sorted_fruits)


['açaí', 'acerola', 'atemoia', 'cajá', 'caju']


In [6]:
import pyuca
coll = pyuca.Collator()
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
soreted_fruits = sorted(fruits, key =coll.sort_key)
print (sorted_fruits)

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']


### 4.8 유니코드 데이터 베이스

In [9]:
import unicodedata
import re

re_digit = re.compile(r'\d')
sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
    print ( 'U+%04x' % ord(char),
            char.center(6),
           're_dig' if re_digit.match(char) else '-',
           'isdig' if char.isdigit() else '-',
           'isnum' if char.isnumeric() else '-',
           format(unicodedata.numeric(char), '5.2f'),
           unicodedata.name(char),
           sep='\t')

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


### 4.9 이중 모드 str 및 bytes API
#### 4.9.1 정규 표현식에서의 str과 API

In [14]:
import re

re_numbers_str = re.compile(r'\d+')
re_words_str = re.compile(r'\w+')
re_numbers_bytes = re.compile(rb'\d+')
re_words_bytes = re.compile(rb'\w+')

text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef"
            " as 1729 = 1³ + 12³ = 9³ + 10³.")

text_bytes = text_str.encode("utf_8")

print ("Text", repr(text_str), sep='\n')
print ('Numbers')
print ('    str  :', re_numbers_str.findall(text_str))
print ('    bytes:', re_numbers_bytes.findall(text_bytes))
print ('Words')
print ('    str  :', re_words_str.findall(text_str))
print ('    bytes:', re_words_bytes.findall(text_bytes))

Text
'Ramanujan saw ௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
    str  : ['௧௭௨௯', '1729', '1', '12', '9', '10']
    bytes: [b'1729', b'1', b'12', b'9', b'10']
Words
    str  : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
    bytes: [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


#### 4.9.2 os 모듈 함수에서 str과 bytes

In [19]:
import os
print (os.listdir('.'))
print (os.listdir(b'.'))

['.ipynb_checkpoints', 'CH1. 파이썬 데이터 모델.ipynb', 'CH.2 시퀀스.ipynb', 'bisect_demo.py', 'floats.bin', 'CH3. 딕셔너리와 집합.ipynb', 'zen.txt', 'CH4. 텍스트와 바이트.ipynb', 'cafe.txt', 'dummy']
[b'.ipynb_checkpoints', b'CH1. \xed\x8c\x8c\xec\x9d\xb4\xec\x8d\xac \xeb\x8d\xb0\xec\x9d\xb4\xed\x84\xb0 \xeb\xaa\xa8\xeb\x8d\xb8.ipynb', b'CH.2 \xec\x8b\x9c\xed\x80\x80\xec\x8a\xa4.ipynb', b'bisect_demo.py', b'floats.bin', b'CH3. \xeb\x94\x95\xec\x85\x94\xeb\x84\x88\xeb\xa6\xac\xec\x99\x80 \xec\xa7\x91\xed\x95\xa9.ipynb', b'zen.txt', b'CH4. \xed\x85\x8d\xec\x8a\xa4\xed\x8a\xb8\xec\x99\x80 \xeb\xb0\x94\xec\x9d\xb4\xed\x8a\xb8.ipynb', b'cafe.txt', b'dummy']


In [24]:
print (os.listdir('.'))
print (os.listdir(b'.'))

pi_name_bytes = os.listdir(b'.')[1]
pi_name_str = pi_name_bytes.decode('ascii', 'surrogateescape')
print (pi_name_str)
print (pi_name_str.encode('ascii', 'surrogateescape'))